In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import mean_squared_error
from scipy.ndimage.interpolation import shift
from math import sin, cos, sqrt, atan2, radians 
from sklearn import tree, svm, linear_model, ensemble, neighbors, naive_bayes 
import dateutil
import os
from pathlib import Path
from joblib import dump, load
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import make_scorer


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
status_path = Path('../filtered_status.csv')
stations_path = Path('../station.csv')

In [3]:
status_df = pd.read_csv(status_path, parse_dates=['time'])
stations_df = pd.read_csv(stations_path)

In [4]:
status_df

,station_id,bikes_available,docks_available,time
0,2,2,25,2013-08-29 12:06:01
1,2,2,25,2013-08-29 12:07:01
2,2,2,25,2013-08-29 12:08:01
3,2,2,25,2013-08-29 12:09:01
4,2,2,25,2013-08-29 12:10:01
...,...,...,...,...
71977905,84,8,7,2015-08-31 23:55:02
71977906,84,8,7,2015-08-31 23:56:01
71977907,84,8,7,2015-08-31 23:57:02
71977908,84,8,7,2015-08-31 23:58:02


In [5]:
stations_df

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
...,...,...,...,...,...,...,...
65,77,Market at Sansome,37.789625,-122.400811,27,San Francisco,8/25/2013
66,80,Santa Clara County Civic Center,37.352601,-121.905733,15,San Jose,12/31/2013
67,82,Broadway St at Battery St,37.798541,-122.400862,15,San Francisco,1/22/2014
68,83,Mezes Park,37.491269,-122.236234,15,Redwood City,2/20/2014


## Ottengo gli id delle stazioni per ogni città diversa

In [4]:
cities = stations_df['city'].unique()
cities

array(['San Jose', 'Redwood City', 'Mountain View', 'Palo Alto',
       'San Francisco'], dtype=object)

In [5]:
city_stations = {}

for city in cities:
    city_stations[city] = stations_df[stations_df['city'] == city]['id'].unique()

city_stations

{'San Jose': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 80, 84]),
 'Redwood City': array([21, 22, 23, 24, 25, 26, 83]),
 'Mountain View': array([27, 28, 29, 30, 31, 32, 33]),
 'Palo Alto': array([34, 35, 36, 37, 38]),
 'San Francisco': array([41, 42, 45, 46, 47, 48, 49, 50, 51, 39, 54, 55, 56, 57, 58, 59, 60,
        61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
        82])}

## Prendo in cosiderazione solo gli id delle stazioni appartenenti alla città di San Francisco

In [6]:
San_Fancisco_stations = city_stations["San Francisco"]
San_Fancisco_stations

array([41, 42, 45, 46, 47, 48, 49, 50, 51, 39, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       82])

In [7]:
interval=30
window_width = 5

## Training

In [8]:
classifiers = {
    'DecisionTree': tree.DecisionTreeClassifier(random_state=42),   
#     'SVC': svm.SVC(),
    'LogisticRegression': linear_model.LogisticRegression(random_state=42, solver='liblinear', max_iter=1000),
    'NaiveBayes': naive_bayes.GaussianNB(),
    'RandomForest': ensemble.RandomForestClassifier(random_state=42, max_features='sqrt'),
}

Effettuo il finetuning dei modelli tramite la grid search

In [ ]:
decision_tree_parameters = {
    'max_depth': list(range(2, 7)),
    'min_samples_split': list(range(2, 7)),
    'criterion' : ['gini', 'entropy'],
    'class_weight': [None, 'balanced']
}

SVC_parameters = {
    'C': [0.1, 1, 100],
    'gamma': [0.1, 0.01, 0.001],
    'kernel': ['rbf'],
    'class_weight': [None, 'balanced']
}

logistic_regression_parameters = {
    'penalty': ['l1', 'l2'],
    'C': [1, 0.1, 0.01, 0.001],
    'class_weight': [None, 'balanced']
}

random_forest_parameters = {
    'n_estimators': [10, 100, 1000],
    'criterion' : ['gini', 'entropy'],
    'max_depth': list(range(0, 5)), 
    'min_samples_split': list(range(0, 5)), 
    'class_weight': [None, 'balanced']
}

naive_bayes_parameters = {
    'var_smoothing': np.logspace(0,-9, num=100)
}

classifiers_parameters = {
  'DecisionTree': decision_tree_parameters,
  'SVC': SVC_parameters,
  'LogisticRegression': logistic_regression_parameters,
  'RandomForest': random_forest_parameters,
  'NaiveBayes': naive_bayes_parameters
}

tuned_classifiers = {
    'DecisionTree': [],
    'SVC': [],
    'LogisticRegression': [],
    'RandomForest': [],
    'KNN': []
}

for reference_station in San_Fancisco_stations:
    train_df = pd.read_csv(f"./datasets/{interval}_{window_width}/station{reference_station}_train.csv")

    y = train_df['status']
    X = train_df.drop(columns=['status'])
    
    for scoring in ['accuracy', 'recall', 'precision', 'f1_score']:
        for classifier in classifiers.keys():
            print(f"Classifier: {classifier}" )   
        
            clf = classifiers[classifier]
            parametrers = classifiers_parameters[classifier]
            
            #I segunti controlli sono necessari in quanto richiamo, precisione e 
            # f1score hanno necessità di conoscere quale sia la classe positiva (QP)
            if scoring == "recall":
                scorer = make_scorer(recall_score, pos_label="QP")
            elif scoring == "precision":
                scorer = make_scorer(precision_score, pos_label="QP")
            elif scoring == "f1_score":
                scorer = make_scorer(f1_score, pos_label="QP")
            else:
                scorer = scoring
            
            #Nel caso di logist regression utilizzo la randomizedSearchCV, in quanto più veloce rispetto alla gridSearchCV
            if classifier == 'LogisticRegression':
                search_result = RandomizedSearchCV(clf, parametrers, scoring=scorer, random_state=42, n_iter=5, n_jobs=-1)
            else:
                search_result = GridSearchCV(clf, parametrers, scoring=scorer, cv=3, n_jobs=-1)
                
            fit_result = search_result.fit(X, y)
            params = fit_result.best_params_

            print("-"*10)
            print(f"Best parameters for classifier {classifier} for the station {reference_station} using the scoring {scoring}: ")
            print(params)
            print()
            print()

            tuned_clf = search_result.best_estimator_
            
            #salvo il modello dopo il fine tuning
            if (Path.cwd() / f'tuned_models/{scoring}/{classifier}/{interval}_{window_width}').exists() == False:
                os.makedirs(f"./tuned_models/{scoring}/{classifier}/{interval}_{window_width}")
            dump(tuned_clf, f'./tuned_models/{scoring}/{classifier}/{interval}_{window_width}/station{reference_station}_tuned_model.joblib')


## Testing dei fine tuned models

In [10]:

results_table = {}

for scoring in ["accuracy", "recall", "precision", "f1_score"]:
    result_matrix = []
    for classifier in classifiers.keys():
        file = open(f'test_results/tuned_classifiers/{classifier}_{scoring}_{interval}_{window_width}_results.txt', "w")
        file.write(f'TESING RESULTS FOR {classifier} CLASSIFIER:\n\n')
        tot_fp = 0
        tot_tp = 0
        tot_fn = 0
        tot_tn = 0

        for station_id in San_Fancisco_stations:
            model = load(f'./tuned_models/{scoring}/{classifier}/{interval}_{window_width}/station{station_id}_tuned_model.joblib')

            test_df = pd.read_csv(f'./datasets/{interval}_{window_width}/station{station_id}_test.csv')
            y_test = test_df['status']
            X_test = test_df.drop(columns=['status'])    

            prediction = model.predict(X_test)
            
            cm = confusion_matrix(y_test, prediction)

            str_= f'{classifier} FOR STATION {station_id}' + '\n'
            str_ += f'Confusion matrix:' + '\n'
            str_ += str(cm) + '\n'

            tn, fp, fn, tp = cm.ravel()
            str_+= f'tp={tp}, fn={fn}, fp={fp}, tn={tn}' +'\n'

            test_accuracy = (tn + tp) / (tn + fp + fn + tp)
#             test_recall = (tp) / (tp + fn)
#             test_precision = (tp) / (tp + fp)
            test_recall = recall_score(y_test, prediction, pos_label='QP')
            test_precision = precision_score(y_test, prediction, pos_label='QP', zero_division=0)
            test_f1_score = f1_score(y_test, prediction, pos_label='QP')

            str_+= f'accuracy={test_accuracy}; recall={test_recall}; precision={test_precision}; f1_score= {test_f1_score}' +'\n\n'
            str_+= "-"*10 +'\n\n'

            tot_fp += fp
            tot_tp += tp
            tot_fn += fn
            tot_tn += tn

            file.write(str_)

        avg_accuracy = (tot_tn + tot_tp) / (tot_tn + tot_fp + tot_fn + tot_tp)
        avg_recall = (tot_tp) / (tot_tp + tot_fn)
        avg_precision = (tot_tp) / (tot_tp + tot_fp)
        avg_f1_score = 2*(1/((1/avg_recall)+(1/avg_precision)))
        
        result_matrix.append([avg_accuracy, avg_recall, avg_precision, avg_f1_score])
        avg_str = f"AVERAGE VALUES FOR {classifier}: accuracy={avg_accuracy}; recall={avg_recall}; precision={avg_precision}; f1_score={avg_f1_score}"

        file.write(avg_str)
        file.close()
    
    result_table = pd.DataFrame(result_matrix, columns=['avg_accuracy', 'avg_recall', 'avg_precision', 'avg_f1_score'], index=pd.Index(classifiers.keys()))
    results_table[scoring] = result_table
    result_table.to_csv(f"./test_results/tuned_classifiers/Overall_results_{scoring}_{interval}_{window_width}.csv")


In [33]:
print("ACCURACY")
results_table["accuracy"].head()

ACCURACY


,avg_accuracy,avg_recall,avg_precision,avg_f1_score
DecisionTree,0.982661,0.727893,0.769046,0.747904
LogisticRegression,0.977276,0.551830,0.738940,0.631823
NaiveBayes,0.872615,0.672008,0.170165,0.271564
RandomForest,0.977401,0.432443,0.857255,0.574885


In [34]:
print("RECALL")
results_table["recall"].head()

RECALL


,avg_accuracy,avg_recall,avg_precision,avg_f1_score
DecisionTree,0.952874,0.929179,0.423879,0.582176
LogisticRegression,0.927497,0.954204,0.322330,0.481880
NaiveBayes,0.822148,0.776261,0.138967,0.235732
RandomForest,0.926847,0.882196,0.311211,0.460110


In [35]:
print("PRECISION")
results_table["precision"].head()

PRECISION


,avg_accuracy,avg_recall,avg_precision,avg_f1_score
DecisionTree,0.982777,0.724332,0.773774,0.748237
LogisticRegression,0.976489,0.521860,0.735946,0.610683
NaiveBayes,0.867572,0.735608,0.174352,0.281891
RandomForest,0.969101,0.269931,0.651468,0.381705


In [36]:
print("F1_SCORE")
results_table["f1_score"].head()

F1_SCORE


,avg_accuracy,avg_recall,avg_precision,avg_f1_score
DecisionTree,0.980736,0.768942,0.709954,0.738272
LogisticRegression,0.969003,0.642038,0.552849,0.594115
NaiveBayes,0.866080,0.750841,0.174948,0.283776
RandomForest,0.956750,0.776855,0.436989,0.559342


In [ ]:
#Codice per ottenere il latex delle diverse tabelle
res_df = pd.read_csv("./test_results/tuned_classifiers/Overall_results_f1_score_30_5.csv", index_col=0)
res_df = res_df.drop(columns=['avg_accuracy'])
print(res_df.to_latex(bold_rows=True))
res_df